In [ ]:
import pandas as pd
import numpy as np
import os

# Lecture 7 – Relational Algebra, Messy Data

## DSC 80, Winter 2023

### Announcements

- Discussion 2 is **tonight at 11:59PM**.
    - To earn 0.3% of extra credit, you must submit Lab 2, attend Discussion 2, and submit the Lab 2 Reflection Form by **Saturday at 11:59PM**.
    - Lab 2 scores and solutions (to non-discussion problems) have been released.
- Project 1 is due **tomorrow at 11:59PM**.
- Lab 3 is due on **Monday, January 30th at 11:59PM**.
- Please fill out the anonymous [Week 3 Feedback Survey](https://docs.google.com/forms/d/e/1FAIpQLSfzz6fxUDf4XA8yPDUsXaIn17wPRfAShYdiPc_6-f3j1VDQHQ/viewform) to let us know how the course has been going so far!

### Agenda

- Relational algebra.
- Working with messy, real-world data.

## Relational algebra

### Birds-eye view of the course

- In just two weeks, we've covered most of the core DataFrame manipulation tools you'll need to extract insight from data.

- In future courses, like DSC 100, you'll revisit how to perform many of the same operations in SQL, a language designed to work with **relational databases**.
    - A relational database is a database that stores tabular data.

- There, you'll learn more about the _theory_ of DataFrames and databases, and will learn about how to think about operations more abstractly using **relational algebra**.

### Relational algebra

- Relational algebra is a system for describing operations that are performed on relations (tables). 

- There are **five _primitive_ relational operators**, each of which produce a new relation.

- Like in regular arithmetic, expressions are made up of values and operators. For instance, the following is an expression in relational algebra:

$$\sigma_{\text{temps.City = countries.City}} \big( \text{temps} \times \text{countries} \big)$$

- Here, we'll give you a brief, imprecise introduction to relational algebra to make the connections more clear in future courses.

For illustration purposes, let's look at the temperatures and countries example DataFrames from the last lecture, with slight modifications.

In [ ]:
temps = pd.DataFrame({
    'City': ['San Diego', 'Toronto', 'Rome'],
    'Temperature': [76, 28, 56],
    'Humid': ['No', 'Yes', 'Yes']
})

other_temps = pd.DataFrame({
    'City': ['Los Angeles', 'San Diego', 'Miami'],
    'Temperature': [79, 76, 88],
    'Humid': ['No', 'No', 'Yes']
})

countries = pd.DataFrame({
    'City': ['Toronto', 'Shanghai', 'San Diego'],
    'Country': ['Canada', 'China', 'USA']
})

In [ ]:
temps

In [ ]:
other_temps

In [ ]:
countries

### Projection ($\Pi$)

Used to project (keep) columns in a relation. Duplicates rows are dropped.

$$\Pi_{(\text{City, Humid})}(\text{temps})$$

In [ ]:
temps[['City', 'Humid']].drop_duplicates()

### Selection ($\sigma$)

Used to keep rows in a relation that satisfy certain conditions.

$$\sigma_{(\text{Temperature} > 50)}(\text{temps})$$

In [ ]:
temps[temps['Temperature'] > 50]

Operators can be composed:

$$\Pi_{(\text{City, Humid})} \big(\sigma_{(\text{Temperature} > 50)}(\text{temps}) \big)$$

In [ ]:
temps.loc[temps['Temperature'] > 50, ['City', 'Humid']].drop_duplicates()

### Cross product ($\times$)

Used to create every possible combination of rows in the first relation with rows in the second relation.

$$\text{temps} \times \text{countries}$$

In [ ]:
# Could also use temps.merge(countries, how='cross').
pd.merge(temps, countries, how='cross')

The cross product is not incredibly useful on its own, but it can be used with other operators to perform more meaningful operations.

What does the following compute?

$$\sigma_{\text{temps.City = countries.City}} \big( \text{temps} \times \text{countries} \big)$$

In [ ]:
both = pd.merge(temps, countries, how='cross')
both[both['City_x'] == both['City_y']]

In [ ]:
temps.merge(countries)

### Union ($\cup$)

Used to combine the rows of two relations. Duplicate rows are dropped. Only works if the two relations have the same attributes (column names).

$$\text{temps} \cup \text{other_temps}$$

In [ ]:
pd.concat([temps, other_temps]).drop_duplicates()

### Difference ($-$)

Used to find the rows that are in one relation but not the other. Only works if the two relations have the same attributes (column names).

$$\text{temps} - \text{other_temps}$$

In [ ]:
temps[~temps['City'].isin(other_temps['City'])]

### Brief summary

- We saw five operators: project ($\Pi$), select ($\sigma$), cross product ($\times$), union ($\cup$), and difference ($-$). 

- You'll learn about more operations in relational algebra in the future, including rename ($\rho$), intersection ($\cap$), join, aggregate, etc. Additional resources: [Wikipedia](https://en.wikipedia.org/wiki/Relational_algebra#Introduction), [UNSW](https://cgi.cse.unsw.edu.au/~cs3311/20T3/lectures/rel-algebra/slides.html#:~:text=Selection%20returns%20a%20subset%20of,satisfy%20a%20specified%20condition%20C.&text=C%20is%20a%20boolean%20expression%20on%20attributes%20in%20R.&text=Projection%20returns%20a%20set%20of,attributes%20in%20the%20original%20relation.), [UCSD DSC 100](https://drive.google.com/file/d/1-txCsFSltz3bCZ1XzzHgZvVEkIOHVbqq/view?usp=sharing).

- **Takeaway**: Think about what each line of code you write is doing – don't just guess-and-check until you pass all `otter` cases.

## Introduction to messy data

### There is no such thing as "raw data"!

* Data are the result of measurements that must be recorded.
* Humans design the measurements and record the results.
* Data is **always** an imperfect record of the underlying processing being measured.

### Data generating process

* A **data generating process** is the underlying, real-world (probabilistic) mechanism that generates observed data. 
* Observed data is an incomplete artifact of the data generating process.
* **A data generating process is what a statistical model attempts to describe.**
    - From DSC 10: a model is a set of assumptions about how data were generated.
    - More on this later in the quarter.
- Data cleaning requires an understanding of the data generating process.

### Example: COVID case counts 🦠

Suppose our **goal** is to determine the number of COVID cases in the US **yesterday**.
- What are we really asking for – the number of people who tested positive yesterday, or the number of people who contracted COVID yesterday?
- Tested positive on what type of test? How accurate is that type of test?
- How often are test results reported? Is there a delay in when test results are reported?

<center><img src='imgs/christmas.png' width=70%></center>

Why do you think so few cases were reported on Christmas Day – is it because COVID was less prevalent on Christmas Day as compared to the days before and after, or is it likely for some other reason? 🎅

### Data provenance

- As data scientists, we often need to work with datasets that others collected, for a purpose that is different than our current interest.
- As such, it's important to understand the "story" of how a dataset came to be, or the **provenance** of the data. Specifically, we need to be aware of:
    1. Assumptions about the data generating process.
    2. How the initial values in the dataset came to be.  
    3. How any data processing or storage decisions affected the values in the dataset.

The bigger picture question we're asking here is, **can we trust our data?**

### Data cleaning 🧹

- Data cleaning is the process of transforming data so that it best represents the underlying data generating process.

- In practice, data cleaning is often detective work to understand data provenance.
    - **Always be skeptical of your data!**

### Keys to data cleaning

Data cleaning often addresses: 

* The **structure** of the recorded data.
    - Is the data stored in a tabular format (e.g. CSV, SQL, Google Sheets) or in another format (JSON, XML)?
    - Are the individuals properly represented as rows?
* The **encoding** and **format** of the values in the data.
    - Are the data types of all columns reflective of the **kinds of data** they contain?
* Corrupt and "**incorrect**" data, and missing values.
    - Were there flaws in the data recording process? In other words, is our data **faithful** to the data generating process?
    
Let's focus on the latter two.

## Kinds of data

### Kinds of data

<center><img src='imgs/data-types.png' width=90%></center>

### Discussion Question

Determine the kind of each of the following variables.
- Fuel economy in miles per gallon.
- Number of quarters at UCSD.
- Class standing (freshman, sophomore, etc.).
- Income bracket (low, medium, high).
- Bank account number.

### Example: DSC 80 students

In the next cell, we'll load in an example dataset containing information about past DSC 80 students.

- `'PID'` and `'Student Name'`: student PID and name.
- `'Month'`, `'Day'`, `'Year'`: date when the student was accepted to UCSD.
- `'2021 tuition'` and `'2022 tuition'`: amount paid in tuition in 2021 and 2022, respectively.
- `'Percent Growth'`: growth between the two aforementioned columns.
- `'Paid'`: whether or not the student has paid tuition for this quarter yet.
- `'DSC 80 Final Grade'`: either `'Pass'`, `'Fail'`, or a number.

What needs to be changed in the DataFrame to extract meaningful insights?

In [ ]:
students = pd.read_csv(os.path.join('data', 'students.csv'))
students

### How much has each student paid in total tuition in 2021 and 2022?

In [ ]:
students

In [ ]:
total = students['2021 tuition'] + students['2022 tuition']
total

### Check the data types of `students`!

* What kinds of data should each column have?
    - Qualitative or quantitative?
    - Discrete or continuous?
    - Ordinal or nominal?
* What data type *should* each column have?

* Use the `dtypes` attribute or the `info` method to peek at the data types.

In [ ]:
students.info()

### Cleaning `'2021 tuition'` and `'2022 tuition'`

* `'2021 tuition'` and `'2022 tuition'` are stored as `object`s (strings), not numerical values.
* The `'$'` character causes the entries to be interpreted as strings.
* We can use `str` methods to strip the dollar sign.
    - Recall, whatever method/operator comes immediately after `.str` will be applied to each element of the Series individually, rather than the Series as a whole.

In [ ]:
# This won't work. Why?
students['2021 tuition'].astype(float)

In [ ]:
# That's better!
students['2021 tuition'].str.strip('$').astype(float)

We can loop through the columns of `students` to apply the above procedure. (Looping through columns is fine, just avoid looping through rows.)

In [ ]:
for col in students.columns:
    if 'tuition' in col:
        students[col] = students[col].str.strip('$').astype(float)
        
students

Alternatively, we can do this without a loop by using `str.contains` to find only the columns that contain tuition information.

In [ ]:
cols = students.columns.str.contains('tuition')
students.loc[:, cols] = students.loc[:, cols].astype(float)
students

### Cleaning `'Paid'`

* Currently, `'Paid'` contains the strings `'Y'` and `'N'`.
    * `'Y'`s and `'N'`s typically result from manual data entry.
* The `'Paid'` column should contain `True`s and `False`s, or `1`s and `0`s.
* One solution: create a Boolean Series through comparison. Could also use the Series `replace` method.

In [ ]:
students['Paid'].value_counts()

In [ ]:
students['Paid'] = students['Paid'] == 'Y'
students

### Cleaning `'Month'`, `'Day'`, and `'Year'`
- Currently, these are stored separately using the `int64` data type. This could be *fine* for certain purposes, but ideally they are stored as a single column (e.g. for sorting).
- Solution: use `pd.to_datetime` to convert dates to `datetime64` objects.

In [ ]:
students.loc[:, 'Month': 'Year']

In [ ]:
students['Date'] = pd.to_datetime(students.loc[:, 'Month': 'Year'])
students = students.drop(columns=['Month', 'Day', 'Year'])
students

### Cleaning `'DSC 80 Final Grade'`

* Currently, `'DSC 80 Final Grade'`s are stored as `object`s (strings).
* Unless we somehow store this column to a numeric type, we can't do any arithmetic with it.
* However, due to the existence of strings like `'Pass'`, we can't use `astype` to convert it.
* Solution: use `pd.to_numeric(s, errors='coerce')`, where `s` is a Series.
    - ⚠️ Be careful with this!
    - `errors='coerce'` can cause uninformed destruction of data.

In [ ]:
# Won't work!
students['DSC 80 Final Grade'].astype(int)

In [ ]:
pd.to_numeric(students['DSC 80 Final Grade'], errors='coerce')

In [ ]:
students['DSC 80 Final Grade'] = pd.to_numeric(students['DSC 80 Final Grade'], errors='coerce')
students

In [ ]:
pd.to_numeric?

### Cleaning `'Student Name'`
* We want names to be formatted as `'Last Name, First Name'`, a common format.
* Solution: use `str` methods once again.

In [ ]:
students['Student Name']

In [ ]:
parts = students['Student Name'].str.split()
parts

In [ ]:
students['Student Name'] = parts.str[1] + ', ' + parts.str[0]
students

### More data type ambiguities

- 1649043031 looks like a number, but is probably a date.
    - As we saw in the last lecture, [Unix timestamps](https://www.unixtimestamp.com) count the number of seconds since January 1st, 1970.

- "USD 1,000,000" looks like a string, but is actually a number **and** a unit.
    
- 92093 looks like a number, but is really a zip code (and isn't equal to 92,093).
    
- Sometimes, `False` appears in a column of country codes. Why might this be? 
🤔

### Example: the Norway problem 🇳🇴

In [ ]:
import yaml

player = '''
name: Magnus Carlsen
age: 32
country: NO
'''

In [ ]:
yaml.safe_load(player)

## Summary, next time

### Summary

- Relational algebra provides a system for describing operations on relations (tables).
- Data provenance describes the "origin story" of a dataset, from the data generating process to its storage.
- Data cleaning is the process of transforming data so that it best represents the underlying data generating process.
- We must ensure that each column in a DataFrame uses the correct data type for the kind of data in the column.

### Next time

- Working with unfaithful data – that is, data that is purely wrong.
- (Re)introduction to hypothesis testing.